# SaildroneEVBuilder: Saildrone data processing from *.raw* files

This notebook goes through the process of:
* Building EV Files
  * Sorting raw files
  * Building EV files from sorted raw file sets
    * GPS filesets
    * Exclusion Lines
    * ST & ET Markers
    * Define Regions
* Exporting $s_{A}$ from EV Files

The processing follows the routines established for the Eastern Bering Sea Saildrone mission established by DeRobertis and M. Levine based off WBAT processing code developed by De Robertis and R. Levine.

# From raw to Macebase2

As descibred in `MASTER_SD_EBS_2016_processing.m`, for each data set (i.e. SD/echosounder combo):
1. Make EV files for scrutinizing (`make_Saildrone_EV_files.m`)
2. Make echogram images (`make_SD_ev_images.m`)
3. **Scrutinizing**: Check bottom and surface exclusion lines
4. Draw regions (`DrawRegions_SD.m`)
5. Export in Macebase2 format (`ExportEchoviewFiles_4_SD`)

But before we even do that, first lets get the filesets all organized....

## 0. EV FIle Data Organizing

In [1]:
#from saildroneParams import paramsSD1022 as processParams # Each saildrone has a specific params file with 
from saildroneParams import paramsSD1023 as processParams # Each saildrone has a specific params file with 
from evFileProcessing import evFuncs # All of the EV file/COM functions are organized in EvFuncs.py within two classes

### .raw files 

We'll read all of the raw files and based on the values defined in the parameters file, organize them into sets of files by size that will determine the number of EV files to make

In [2]:
rawFileGroups= evFuncs.evFiles.evRawFileSorter(processParams)

Found 5966 .raw files 
First Wakeup: D20180618-T184913
Last Wakeup: D20181005-T191202
Total deployment: 98 days 
Mission days for EV files: 4
Grouping results in 5 EV files 
We skipped  5775 raw files


### GPS files

Because the saildrone raw files do not contain the GPS data, there is a separate fileset in csv format that we need to also group into appropriate filegroups in a list to provide to Echoview along with each raw filegroup

There are 4 ways that a gps file could fit with a raw file group.  Lets think of it like this:

| Scenario                       |          |               |          |       |          |             |        |
|--------------------------------|----------|---------------|----------|-------|----------|-------------|--------|
|                                |          | StartRawGroup | >>>>>    | >>>>> | >>>>>    | EndRawGroup |        |
| 1 - Starts before, ends within | startGPS | >>>>>         | endGPS   |       |          |             |        |
| 2 - Starts within, ends within |          |               | startGPS | >>>>> | endGPS   |             |        |
| 3 - Starts within, ends after  |          |               |          |       | startGPS | >>>>>       | endGPS |
| 4 - Starts before, ends after  | startGPS | >>>>>         | >>>>>    | >>>>> | >>>>>    | >>>>>       | endGPS |

In [3]:
gpsFileGroups = evFuncs.evFiles.evGPSFileSorter(rawFileGroups,processParams)

We have 5 GPS groupings to go with our 5 raw file groups


Now there should be two complete file groups:
- `rawFileGroups`: List where each element itself is a list of `*.raw` files (sorted by date) whose summed file sizes are <= the value specified in the parameters file
- `gpsFileGroups`: List where the *i-th* is a list of `*.gps.csv` files which bound the dates of the raw files in the *i-th* numbered group in `rawFileGroups`

## 1. Make EV Files for Scrutinizing

In [ ]:
# To build the ev filces:
evFiles = evFuncs.evFiles.evBuilder(rawFileGroups,gpsFileGroups, processParams)

## 2. Make echogram images

In [ ]:
# To build the ev images:
imageFiles = evFuncs.evFiles.evImages(rawFileGroups, processParams)

## 3. Scrutinizing: Check bottom and surface exclusion lines
Well, go do it already!

### 3.1 I forgot to put in a variable combining the frequency response and the final sA so...

I wanted to be able to look at the sA after the 200 frequency response filtering. We ended up not using this variable but it wasn't in the original template.

In [ ]:
from glob import glob
import time
import numpy as np
evFiles = glob('../../../data/EV/EVFiles/SD1022/Reviewed/*.EV')
#evFiles = glob('D:/AIESII/Saildrone/EV/EVFiles/SD1022/Reviewed/*.EV')
import win32com.client
EvApp = win32com.client.Dispatch("EchoviewCom.EvApplication")
EvApp.Minimize()
for file in evFiles:
    print('Working on file: '+file)
    start = time.time()
    EvFile = EvApp.OpenFile(file)
    EvVar38 = EvFile.Variables.FindByName('final SD data_reduced pings')
    EvVarNew= EvVar38.AddVariable(3)
    EvVarNew.SetFullName("38 kHz Reduced w/FR Filter")
    frVar = EvFile.Variables.FindByName('-14 < FR < 2 dB')
    a = EvVarNew.SetOperand(2,frVar)
    EvFile.Save()
    EvFile.Close()
    print('File Saved. Total time: ' + str(int(np.floor((time.time()-start)/60)))+'m '+str(round((time.time()-start)%60))+ 's')

### 3.2 Correcting TS in Sawada Formula Operator

My original exports used a -55 TS (5cm arctic cod) for the Sawada index calculation. I wanted to update the formula to use -57.3 which corresponds to a 3.5cm arctic cod. The way to do this for the old=style EV formula operator is to replace the variable with an imported variable from a different template, in this case, a new EV file called `SD_2018_sawadaCorrection` whichonly contains 1 variable, the corrected formula.

In [ ]:
from glob import glob
import time
import numpy as np3
evFiles = glob('../../../data/EV/EVFiles/SD1022/Reviewed/*.EV')
#evFiles = glob('D:/AIESII/Saildrone/EV/EVFiles/SD1022/Reviewed/*.EV')
import win32com.client
EvApp = win32com.client.Dispatch("EchoviewCom.EvApplication")
EvApp.Minimize()
for file in evFiles:
    print('Working on file: '+file)
    start = time.time()
    EvFile = EvApp.OpenFile(file)
    EvFile.Import('../../../data/EV/SD_2018_sawadaCorrection.EV'')
    VarParent1 = EvFile.Variables.FindByName("38 kHz Sv in cell")
    VarChild1 = EvFile.Variables.FindByName("Nv threshold bitmap")
    VarNew = EvFile.Variables.FindByName("Fish per volume in cell corr")
    VarNew.SetOperand(1, VarParent1)
    VarChild1.SetOperand(1, VarNew)
    EvFile.Save()
    EvFile.Close()
    print('File Saved. Total time: ' + str(int(np.floor((time.time()-start)/60)))+'m '+str(round((time.time()-start)%60))+ 's')
EvApp.Quit()

## 4.1 Export in Macebase2 format

This function has been modified to export 38 kHz at 5m, 1m vertical resolution, and sawada filtered targets, so that the file only has to be integrated once.

In [ ]:
# To export the ev files
from saildroneParams import paramsSD1022 as processParams # Each saildrone has a specific params file with 
from evFileProcessing import evFuncs # All of the EV file/COM functions are organized in EvFuncs.py within two classes
from glob import glob
evFiles = glob('../../../data/EV/EVFiles/SD1022/Reviewed/*.EV')
#evFiles = glob(processParams.evParams.outputDir+'*.EV')
exportFiles = evFuncs.evFiles.evExporter(evFiles, processParams)

In [ ]:
# To export the ev files
from saildroneParams import paramsSD1023 as processParams # Each saildrone has a specific params file with 
from evFileProcessing import evFuncs # All of the EV file/COM functions are organized in EvFuncs.py within two classes
from glob import glob
evFiles = glob('../../../data/EV/EVFiles/SD1023/Reviewed/*.EV')
#evFiles = glob(processParams.evParams.outputDir+'*.EV')
exportFiles = evFuncs.evFiles.evExporter(evFiles, processParams)

## 4.3 Export Targets

To separately export targets.

In [34]:
import win32com.client
from glob import glob
from saildroneParams import paramsSD1022 as processParams1022 # Each saildrone has a specific params file with 
from saildroneParams import paramsSD1023 as processParams1023 # Each saildrone has a specific params file with 
from evFileProcessing import evFuncs # All of the EV file/COM functions are organized in EvFuncs.py within two classes

#evFiles1022 = glob('E:/AIESII/Saildrone/EV/EVFiles/SD1022/Reviewed/*.EV')
#evFiles1022 = glob('D:/AIESII/Saildrone/EV/EVFiles/SD1022/Reviewed/*.EV')
evFiles1023 = glob('../../../data/EV/EVFiles/SD1023/Reviewed/*.EV')

evFuncs.evFiles.evTargetExporter([evFiles1023,processParams1023)

File 2 Exported. Total time: 21m 11s
Done


['E:\\AIESII\\Saildrone\\Submission\\data\\EV\\EVFiles\\SD1023\\exports\\SingleTargetsNv-1023-5m-20180715-20180716.csv']